# Meta Programming

## Top Summary 
Metaprogramming is a term often used, but probably is seldom a
target. It is a general description for techniques that manipulate
code at run-time. Decorators, Metaclasses and Descriptors are all
part of metaprogramming - and for Python that list is not
exhaustive.
You might never use these techniques, but you almost certainly
will use modules or products that do. 

We have already experienced ,some of this previously with decorators 

Metaprogramming is used in Python for a variety of tasks, including:

Creating dynamic class definitions and instances: Metaclasses can be used to create new classes dynamically at runtime, allowing for more flexible and customizable class structures.

Modifying the behavior of functions and classes: Decorators can be used to modify the behavior of functions and classes, adding functionality such as caching, logging, or validation.

Can be used for debugging

Creating domain-specific languages (DSLs) or frameworks: Metaprogramming can be used to create domain-specific languages, which are specialized languages designed for a particular application domain.


IN this sections we wil look at 
- Monkey-patching
- Descriptors
- Signatures
- Class decorators
- Metaclasses
- Abstract Base Classes
- Virtual subclasses
- __call__

# monkey-patching

Monkey-patching is the altering of existing classes by replacement.
That is easy in Python, but you might say "too easy". Beginners to
Python often call variables names like len, list, or str, and the most
common overwrite in Python 2 is storing a filename in a variable
called file (an alias of open). Cant do that in Python 3
There are obvious temptations to hack. The problem is that code
might not behave in the way users expect - changing functionality
to that documented is not always desirable, and can be
unsupportable. Perhaps inheritance would be worth a try instead?


## descriptors
Introduced in Python 2.2 Descriptors are used to implement the underlying functions of the object system which include bound and unbound methods, class methods, and static methods. 

A lot of modern frameworks and libraries use the "descriptor" protocol to make the process of creating APIs for end-users neat and simple. Python's builtins like property, staticmethod and classmethod can be imitated using the descriptor protocol.

Consider the following example class:

For a class to be a descriptor it hs to implement the descriptor protocol The descriptor protocol the following functions may be implemented 
`__get__ (self, obj, type=None)`
Used to access attributes. It returns the value of the attribute. If the attribute is illegal, it can throw a corresponding exception like ValueError. If the attribute does not exist, it will report AttributeError.

`__set__ (self, obj, value)`
Used to set the property’s values, none will be returned.

`__delete__ (self, obj)`
Controls the deletion of attributes; none will be returned.

In [22]:
class ValidatedEngine:    
    def __get__(self, obj, objtype = None):
        value = obj._size
        return value
    
    def __set__(self,obj,size):
        if not  1 <= size <= 6:
            raise ValueError('Engine size must be in range 1 to 6')
        obj._size = size

class Car:
    engine = ValidatedEngine()
    
    def __init__(self, model, engine):
        self.model = model
        self.engine = engine
        
    def upgrade_engine(self, size):
        self.engine += size

        

In [23]:
my_car = Car("BMW", 2.0)



In [24]:
my_car.engine

2.0

In [25]:
my_car.upgrade_engine(10)

ValueError: Engine size must be in range 1 to 6

## Signatures
Signatures are relatively new, and they represent the call interface
of a function. 

In Python, Signature and Parameter objects are created by the
inspect module. A Signature is immutable, but you can create a
new one using Signature.replace().
A Signature has a parameters’ attribute, which is a container
(mapping) of Parameter objects.

A Signature object represents a function's parameter list
- It is a container of Parameter objects
- Can yield a list of parameter names


Signature and Parameter are from the inspect module

You can list and create signatures for methods/functions
- Signatures are created from a container of Parameters
- Parameters are created from a string name and a type
Example type: POSITIONAL_OR_KEYWORD
A Signature object can be associated with a function using:
Signature.bind(*args. *kwargs)


the following code draws from an earlier example of setattr and this 
could be antoher way of doing this using signature

Set struct is a base class that implements an empty `__signature__` class attribute 
unrelated classes can inherit from the common base class below


In [28]:
from inspect import Parameter, Signature

def create_sig(fields):
    params = []
    for pname in fields:
        params.append(Parameter(pname, Parameter.POSITIONAL_OR_KEYWORD))
    return Signature(params)

class SetStruct:
    __signature__ = create_sig([])
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for attr_name, attr_val in bound.arguments.items():
            setattr(self, attr_name, attr_val)

    def __str__(self):
        lstr = []
        for pname in self.__signature__.parameters:
            lstr.append(str(getattr(self, pname, '')))
        return ', '.join(lstr)

    

Lets set up two unrelated classes using that signature

In [29]:
class File(SetStruct):
    __signature__ = create_sig(['filename', 'size', 'owner'])
    
class Person(SetStruct):
    __signature__ = create_sig(['name', 'address', 'dob'])


In [31]:
print(File.__signature__)

print(Person.__signature__)


f1 = File('/usr/lib/python', 123, 'root')
f2 = File('/home/user1/country.txt', 234, 'user1')
f3 = File(owner = 'user2', filename = '/home/user2/gash',size = 1)


(filename, size, owner)
(name, address, dob)


In [32]:
print(f1)

/usr/lib/python, 123, root


In [33]:
p1 = Person('Fred Bloggs', '4 Railway Cuttings', '12/12/1990')
p2 = Person('Jim James', '123 Acacia Avenue', '4/5/1988')
p3 = Person('Mary Lamb', '4 The Field', '1/1/1970')


In [34]:
print(p1)

Fred Bloggs, 4 Railway Cuttings, 12/12/1990


## Class decorators

Unlike function decorators, class decorators return a class object.
We shall be looking at metaclasses soon, but class decorators are a
simpler way of doing a similar thing - such as some extra
processing when a class is defined (metaclasses can do more
though).
One use is to provide a new `__getattribute__` method, which might
provide additional functionality or logging.




A metaclass is a class that creates classes

You use a metaclass every time you create a new class, the devault metaclass in python is Type.
Even classes are objects, and they are created by a metaclass. While the
Python system provides such classes transparently, you can
provide your own. They are typically used in products such as
frameworks and GUI debuggers to keep track of classes.
In a class hierarchy you could consider the super class to be a
metaclass.

In [1]:
class MyDecorator:
    def __init__(self, function):
        self.function = function

    def __call__(self, *args, **kwargs):
        print("__call__ was called, this means it was callable")
        self.function(*args, **kwargs)



# adding class decorator to the function
@MyDecorator
def function(person, status):
    print(f"{person} {status}")


function("Phil", "Running for his life")


__call__ was called, this means it was callable
Phil Running for his life


## Abstract Base Class
An Abstract Base Class (ABC) is a class which cannot be directly
instantiated, but can be derived. Abstract methods described in
the ABC must be implemented in the derived class, otherwise
objects of that class cannot be instantiated either.


Using ABC  forces an interface that classes must implement, this allows for variation.

The follwong is an example of Animal class using abstract method notice that the last one does not follow
the pattern

In [36]:
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def make_sound(self):
        pass

class Dog(Animal):
    def make_sound(self):
        return "Woof!"

class Cat(Animal):
    def make_sound(self):
        return "Meow!"

class Cow(Animal):
    pass


In [38]:
animals = [Dog(), Cat(), Cow()]
for animal in animals: 
    print(animal.make_sound())

TypeError: Can't instantiate abstract class Cow with abstract method make_sound

## __call__
This special method enables the class to be used as if it was a
function - it makes the class callable. 

Its useful in many ways , such as when it is needed to define a callable object that 
maintains some internal state, for example if there is a process that requires multiple steps and
it would be useful to encapsulate that behaviour

Additionally it could be used in debugging : 


In [39]:
class debug:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print(f"Calling function {self.func.__name__} with args={args} kwargs={kwargs}")
        result = self.func(*args, **kwargs)
        print(f"Function returned {result}")
        return result

@debug
def add(x, y):
    return x + y

print(add(2, 3))  


Calling function add with args=(2, 3) kwargs={}
Function returned 5
5
